In [33]:
import cv2
import mediapipe as mp
import util
import pyautogui

In [32]:
pip install pyautogui

  Using cached pyautogui-0.9.54-py3-none-any.whl
  Using cached pymsgbox-2.0.1-py3-none-any.whl.metadata (4.4 kB)
  Using cached pytweening-1.2.0-py3-none-any.whl
  Using cached pyscreeze-1.0.1-py3-none-any.whl
  Using cached pygetwindow-0.0.9-py3-none-any.whl
  Using cached mouseinfo-0.1.3-py3-none-any.whl
  Using cached pyrect-0.2.0-py2.py3-none-any.whl
  Using cached pyperclip-1.11.0-py3-none-any.whl.metadata (2.4 kB)
Using cached pymsgbox-2.0.1-py3-none-any.whl (10.0 kB)
Using cached pyperclip-1.11.0-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x *)

In [ ]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    
    return None


In [ ]:
def detect_gestures(frame , landmarks_list , processed):
    if len(landmarks_list)>= 21:
        
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance(landmarks_list[4], landmarks_list[5])
        
        if thumb_index_dist< 50 and util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])>90:
            move_mouse() 

In [34]:
def main():
    cap = cv2.VideoCapture(0)

    mpHands = mp.solutions.hands
    hands = mpHands.Hands(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7,
        max_num_hands=1
    )

    draw = mp.solutions.drawing_utils

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            processed = hands.process(frameRGB)

            landmark_list = []

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmark_list.append((lm.x, lm.y))
                    
            detect_gestures(frame, landmark_list , processed)


                # print(landmark_list)  use this print statement if you want to check score for the had
 
            cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()


if __name__ == '__main__':
    main()


KeyboardInterrupt: 